# Modeling (GARCH, VAR/Granger/IRFs, Wavelets)
This notebook fits GARCH(1,1) per asset, a VAR model for BTC↔S&P spillovers, and a discrete wavelet transform (DWT) for scale analysis.

**Outputs saved:**
- `figures/*_garch_sigma.png`
- `tables/garch_params.csv`
- `figures/var_impulse_responses.png`, `tables/var_summary.txt`
- `figures/wavelet_energy.png`, `tables/wavelet_energy.csv`


In [9]:
import os
from google.colab import files

DATA = "/content/data"
os.makedirs(DATA, exist_ok=True)

print("Choose your merged_returns.csv file…")
up = files.upload()
fname = next(iter(up))
os.replace(f"/content/{fname}", f"{DATA}/merged_returns.csv")
print("Saved to:", f"{DATA}/merged_returns.csv")


⬆️ Choose your merged_returns.csv file…


Saving merged_returns.csv to merged_returns.csv
✅ Saved to: /content/data/merged_returns.csv


In [10]:
!pip -q install arch statsmodels pandas numpy matplotlib

import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
from arch import arch_model
from statsmodels.stats.diagnostic import acorr_ljungbox

BASE = "/content" if os.path.exists("/content") else "."
DATA = f"{BASE}/data"
FIG  = f"{BASE}/figures"; os.makedirs(FIG, exist_ok=True)
TAB  = f"{BASE}/tables";  os.makedirs(TAB, exist_ok=True)

csv_path = os.path.join(DATA, "merged_returns.csv")
df = pd.read_csv(csv_path, parse_dates=["Date"]).set_index("Date").sort_index()

series_map = {}
if "BTC_Close_Return" in df.columns:
    series_map["BTC-USD"] = df["BTC_Close_Return"]
elif "BTC_Close" in df.columns:
    series_map["BTC-USD"] = np.log(df["BTC_Close"]).diff() * 100
if "ETH_Close_Return" in df.columns:
    series_map["ETH-USD"] = df["ETH_Close_Return"]
if "SPX_Close_Return" in df.columns:
    series_map["^GSPC"] = df["SPX_Close_Return"]
elif "SPX_Close" in df.columns:
    series_map["^GSPC"] = np.log(df["SPX_Close"]).diff() * 100
if "DJI_Close_Return" in df.columns:
    series_map["^DJI"] = df["DJI_Close_Return"]

rets = pd.DataFrame(series_map).dropna(how="all").dropna()
targets = [c for c in ["BTC-USD","ETH-USD","^GSPC","^DJI"] if c in rets.columns] or list(rets.columns)[:4]

rows = []
for col in targets:
    y = rets[col].dropna()
    am = arch_model(y, vol="GARCH", p=1, q=1, mean="Constant", dist="normal")
    res = am.fit(disp="off")
    p = res.params
    alpha = p.get("alpha[1]"); beta = p.get("beta[1]")
    lb_p10 = acorr_ljungbox(res.std_resid.dropna(), lags=[10], return_df=True)["lb_pvalue"].values[0]
    rows.append({
        "series": col, "mu": p.get("mu"), "omega": p.get("omega"),
        "alpha": alpha, "beta": beta, "alpha_plus_beta": (alpha+beta),
        "loglik": res.loglikelihood, "ljungbox_p(10)": lb_p10
    })
    res.conditional_volatility.plot()
    plt.title(f"GARCH(1,1) Conditional Volatility — {col}")
    plt.xlabel("Date"); plt.ylabel("Volatility (σ_t)")
    plt.tight_layout(); plt.savefig(f"{FIG}/{col.replace('^','')}_garch_sigma.png", dpi=150); plt.close()

garch_df = pd.DataFrame(rows)
garch_df.to_csv(f"{TAB}/garch_params.csv", index=False)

def to_latex_table(df, caption="Estimated GARCH(1,1) parameters", label="tab:garch_params"):
    order = ["series","mu","omega","alpha","beta","alpha_plus_beta","loglik","ljungbox_p(10)"]
    use = [c for c in order if c in df.columns]
    return df[use].to_latex(index=False, float_format="%.4f", caption=caption, label=label, escape=False)
with open(f"{TAB}/garch_params.tex","w") as f:
    f.write(to_latex_table(garch_df))

print("Saved:", f"{TAB}/garch_params.csv", "and", f"{TAB}/garch_params.tex")
garch_df.head()


/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.001916. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.003335. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning:

Saved: /content/tables/garch_params.csv and /content/tables/garch_params.tex


,series,mu,omega,alpha,beta,alpha_plus_beta,loglik,ljungbox_p(10)
0,BTC-USD,0.002554,0.000150,0.156217,0.788258,0.944476,1768.605904,0.421728
1,ETH-USD,0.002983,0.000079,0.155447,0.844184,0.999631,1536.863053,0.323482
2,^GSPC,0.000760,0.000004,0.199989,0.780015,0.980004,3089.420072,0.383133
3,^DJI,0.000558,0.000004,0.199998,0.780003,0.980000,3173.449547,0.452668


In [12]:
import os, glob, zipfile
from google.colab import files

BASE = "/content" if os.path.exists("/content") else "."
FIG  = f"{BASE}/figures"
os.makedirs(FIG, exist_ok=True)

patterns = [
    "figure1_volatility.png",
    "rolling_corr_btc_spx.png",
    "normalized_prices.png",
    "corr_heatmap.png",
    "hist_*.png",
    "acf_*.png",
    "pacf_*.png",
    "*_garch_sigma.png",
    "var_impulse_responses.png",
    "wavelet_energy.png",
]

selected = []
for pat in patterns:
    selected.extend(glob.glob(os.path.join(FIG, pat)))

if not selected:
    raise FileNotFoundError(
        f"No matching figures found in {FIG}. "
        "Run the EDA/modeling cells first to generate plots."
    )

zip_path = os.path.join(BASE, "draft_figures.zip")
with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    for p in sorted(set(selected)):
        arcname = os.path.join("figures", os.path.basename(p))
        zf.write(p, arcname=arcname)

print(f"Zipped {len(set(selected))} figures → {zip_path}")
files.download(zip_path)


Zipped 4 figures → /content/draft_figures.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
from google.colab import files
files.download("/content/tables/garch_params.csv")
files.download("/content/tables/garch_params.tex")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR

BASE = "/content" if os.path.exists("/content") else "."
DATA = f"{BASE}/data"
FIG  = f"{BASE}/figures"; os.makedirs(FIG, exist_ok=True)

csv_path = f"{DATA}/merged_returns.csv"
df = pd.read_csv(csv_path, parse_dates=["Date"]).set_index("Date").sort_index()

series = {}
if "BTC_Close_Return" in df: series["BTC"] = df["BTC_Close_Return"]
elif "BTC_Close" in df:      series["BTC"] = np.log(df["BTC_Close"]).diff()*100
if "SPX_Close_Return" in df: series["SPX"] = df["SPX_Close_Return"]
elif "SPX_Close" in df:      series["SPX"] = np.log(df["SPX_Close"]).diff()*100

Y = pd.DataFrame(series).dropna()
assert {"BTC","SPX"} <= set(Y.columns), "Need BTC and SPX return series."

res = VAR(Y).fit(ic="aic", trend="c")
print("AIC-selected lag:", res.k_ar)

with open(f"{BASE}/tables/var_summary.txt", "w") as f:
    f.write(str(res.summary()))
    try:
        f.write("\n\nGranger (BTC <- SPX):\n")
        f.write(str(res.test_causality("BTC", ["SPX"], kind="f").summary()))
        f.write("\n\nGranger (SPX <- BTC):\n")
        f.write(str(res.test_causality("SPX", ["BTC"], kind="f").summary()))
    except Exception as e:
        f.write(f"\n\nGranger tests error: {e}")

irf = res.irf(10)
fig = irf.plot(orth=False)
out = f"{FIG}/var_impulse_responses.png"
fig.savefig(out, dpi=150); plt.close(fig)
print("Saved:", out)


/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


AIC-selected lag: 9
Saved: /content/figures/var_impulse_responses.png


In [16]:
import os, zipfile
from google.colab import files

BASE = "/content" if os.path.exists("/content") else "."
FIG  = f"{BASE}/figures"
TAB  = f"{BASE}/tables"

targets = [
    os.path.join(FIG, "var_impulse_responses.png"),
    os.path.join(TAB, "var_summary.txt"),
]

present = [p for p in targets if os.path.exists(p)]
missing = [p for p in targets if not os.path.exists(p)]
print("Found:", [os.path.basename(p) for p in present])
if missing:
    print("Missing:", [os.path.basename(p) for p in missing],
          "\n(Generate them first by running the VAR cell.)")

zip_path = os.path.join(BASE, "var_results.zip")
with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    for p in present:
        arcname = f"{'figures' if '/figures/' in p else 'tables'}/{os.path.basename(p)}"
        zf.write(p, arcname=arcname)

print(f"Zipped {len(present)} file(s) → {zip_path}")
files.download(zip_path)

Found: ['var_impulse_responses.png', 'var_summary.txt']
Zipped 2 file(s) → /content/var_results.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
!pip -q install PyWavelets pandas numpy matplotlib

import os, numpy as np, pandas as pd, matplotlib.pyplot as plt, pywt

BASE = "/content" if os.path.exists("/content") else "."
DATA = f"{BASE}/data"
FIG  = f"{BASE}/figures"; os.makedirs(FIG, exist_ok=True)
TAB  = f"{BASE}/tables";  os.makedirs(TAB, exist_ok=True)

rets_path = f"{DATA}/returns_daily.csv"
merged_path = f"{DATA}/merged_returns.csv"

def load_returns():
    if os.path.exists(rets_path):
        rets = pd.read_csv(rets_path, parse_dates=["Date"]).set_index("Date").sort_index()
        return rets
    if not os.path.exists(merged_path):
        raise FileNotFoundError("Place either data/returns_daily.csv or data/merged_returns.csv in the data/ folder.")
    df = pd.read_csv(merged_path, parse_dates=["Date"]).set_index("Date").sort_index()
    out = {}
    if "BTC_Close_Return" in df.columns:
        out["BTC-USD"] = df["BTC_Close_Return"]
    elif "BTC_Close" in df.columns:
        out["BTC-USD"] = np.log(df["BTC_Close"]).diff() * 100
    if "SPX_Close_Return" in df.columns:
        out["^GSPC"] = df["SPX_Close_Return"]
    elif "SPX_Close" in df.columns:
        out["^GSPC"] = np.log(df["SPX_Close"]).diff() * 100
    if "ETH_Close_Return" in df.columns:
        out["ETH-USD"] = df["ETH_Close_Return"]
    if "DJI_Close_Return" in df.columns:
        out["^DJI"] = df["DJI_Close_Return"]
    rets = pd.DataFrame(out).dropna(how="all")
    if rets.empty:
        raise ValueError("No usable returns found. Provide BTC returns/close in merged_returns.csv or use returns_daily.csv.")
    return rets

rets = load_returns()

series_name = "BTC-USD" if "BTC-USD" in rets.columns else rets.columns[0]
x = rets[series_name].dropna().values

wavelet = "db4"
level = 3

coeffs = pywt.wavedec(x, wavelet=wavelet, level=level)
energies = [float(np.sum(c**2)) for c in coeffs]
labels   = [f"A{level}"] + [f"D{j}" for j in range(level, 0, -1)]

df_energy = pd.DataFrame({"component": labels, "energy": energies})
df_energy.to_csv(f"{TAB}/wavelet_energy.csv", index=False)

plt.figure(figsize=(7.5,4.5))
plt.bar(labels, energies)
plt.title(f"Wavelet Energy — {series_name} ({wavelet}, level={level})")
plt.xlabel("Component"); plt.ylabel("Energy")
out_png = f"{FIG}/wavelet_energy.png"
plt.tight_layout(); plt.savefig(out_png, dpi=150); plt.close()

print("Saved:")
print(" -", out_png)
print(" -", f"{TAB}/wavelet_energy.csv")


Saved:
 - /content/figures/wavelet_energy.png
 - /content/tables/wavelet_energy.csv


In [18]:
import os, zipfile
from google.colab import files

BASE = "/content" if os.path.exists("/content") else "."
FIG  = f"{BASE}/figures"
TAB  = f"{BASE}/tables"

wavelet_fig = os.path.join(FIG, "wavelet_energy.png")
wavelet_csv = os.path.join(TAB, "wavelet_energy.csv")

present = [p for p in (wavelet_fig, wavelet_csv) if os.path.exists(p)]
missing = [p for p in (wavelet_fig, wavelet_csv) if not os.path.exists(p)]

print("Found:", [os.path.basename(p) for p in present])
if missing:
    print("Missing:", [os.path.basename(p) for p in missing],
          "\n(Generate them first by running the Wavelet cell.)")

zip_path = os.path.join(BASE, "wavelet_results.zip")
with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    for p in present:
        arcname = f"{'figures' if '/figures/' in p else 'tables'}/{os.path.basename(p)}"
        zf.write(p, arcname=arcname)

print(f"Zipped {len(present)} file(s) → {zip_path}")
files.download(zip_path)

Found: ['wavelet_energy.png', 'wavelet_energy.csv']
Zipped 2 file(s) → /content/wavelet_results.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>